# Шаг 1. Получение, осмотр и объединение данных

Основной датасет.
entry_date — дата записи;
order_id — идентификационный номер заказа;
customer_id — идентификационный номер клиента;
quantity — количество;
price — цена;
name_clust — автоматически присвоенная группа записи на основе названия;
entry_id — идентификационный номер записи;
country_id — идентификационный номер страны.


Текстовое описание записей.
entry_id — идентификационный номер записи;
entry — запись.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
from scipy.stats import shapiro, levene
from scipy.stats import kruskal
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import mannwhitneyu
from io import BytesIO
import requests
from itertools import combinations
from collections import defaultdict

In [ ]:
df = pd.read_csv('/datasets/gift.csv')

In [ ]:
df.info()

таблица содержит 356940 строк и не имеет пропусков, все столбцы имеют корректный тип данных, кроме колонки с датой, это необходимо исправить

In [ ]:
df['entry_date'] = pd.to_datetime(df['entry_date'], format="%d/%m/%Y %H:%M")

In [ ]:
df.head()

In [ ]:
df.info()

загрузим вторую таблицу и рассмотрим информацию о ней 

In [ ]:
spreadsheet_id = '1KzgGARYw8uxBpH-q_rWaR8wg5tMgdVCRH_LNY1idjG0'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
df_gift = pd.read_csv(BytesIO(r.content))
df_gift

In [ ]:
df_gift.info()

для удобства использования таблицы установим столбец с id в качестве индекса

In [ ]:
df_gift = df_gift.set_index('entry_id')

таблица содержит 1 пропуск и так как столбец содержащий его хранит информацию с описанием заказа, заменим его на прочерк

In [ ]:
df_gift['entry'].fillna('-', inplace=True)

In [ ]:
df_gift.head()

проверим наличие дубликатов в таблицах

In [ ]:
df.duplicated().sum()

основная таблица содержит 3573 дубликата, рассмотрим соответствует ли количесво уникальных id заказов их общему числу

In [ ]:
df['order_id'].nunique() == df['order_id'].count()

In [ ]:
#число уникальных идентификаторов
df['order_id'].nunique()

In [ ]:
value_order = df['order_id'].value_counts()
value_order

в таблице есть множество заказов с одинаковым номером 

In [ ]:
#найдем эти заказы 
not_unique_id = value_order[value_order > 1].index

# Фильтруем DataFrame по этим id
duplicates = df[df['order_id'].isin(not_unique_id)]
duplicates

у всех этих заказов совпадает время оформления, но отличается состав 

In [ ]:
#рассмотрим заказы с одинаковым id и временем
same_date_id = duplicates[duplicates.duplicated(['order_id', 'entry_date'], keep=False)]
grouped = same_date_id.groupby(['order_id', 'entry_date'])

# Проверка различий в составе заказов
for (order_id, time), group in grouped:
    if len(group.drop_duplicates()) > 1:
        print(f"Разные составы для order_id {order_id}, время {time}:")
        print(group)

таких заказов огромное количество и неясно чем вызваны эти дубликаты, либо это особенность бизнес процесса, который разбивает большие заказы на части, либо ошибка при сборе данных

на данный момент оставим эти записи и удалим только полные дубликаты

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

In [ ]:
#посчитаем дубликаты во второй таблице
df_gift.duplicated().sum()

во второй таблице дубликаты отсутствуют

In [ ]:
#объеденим информацию в одну таблицу, для этого воспользуемся левим присоединением тк нам нужны все строки из первой таблицы и соответствующие им во второй
df_gift_info = df.merge(df_gift, on='entry_id', how='left')

In [ ]:
df_gift_info.info()

# Шаг 2. Предобработка и начало исследовательского анализа

## рассмотрим выбросы в столбцах quantity и price

In [ ]:
df_gift_info['price'].describe()

In [ ]:
#изменим формат вывода чисел
stats = df_gift_info['price'].describe().apply(lambda x: f"{x:.2f}")
stats

In [ ]:
sns.boxplot(df_gift_info['price'])

в столбце содержаться минусовые значения чего быть не может, а также подозрительно огромные значения 

In [ ]:
#рассмотрим виды товаров магазина
df_gift_info['entry'].unique()

как известно из информации о клиенте, что это магазин подарочных товаров  и вероятность того что какой-либо товар может стоить больше миллиона крайне мала, поэтому оставим верхнюю границу цены на товар 3000 

In [ ]:
filtered_df = df_gift_info[(df_gift_info['price'] > 0) & (df_gift_info['price'] < 3000)]

filtered_df['price'].describe().apply(lambda x: f"{x:.2f}")

In [ ]:
sns.boxplot(filtered_df['price'])

рассмотрим столбец quantity

In [ ]:
df_gift_info['quantity'].describe()

In [ ]:
sns.boxplot(df_gift_info['quantity'])

в столбце также присутствуют отрицательные значения и крайне огромные 

сказано что магазин работает с оптовыми покупателями, поэтому оставим верхнуюю границу в размере 30 товаров

In [ ]:
filtered_df = df_gift_info[(df_gift_info['quantity'] > 0) & (df_gift_info['quantity'] < 30)]

filtered_df['quantity'].describe()

In [ ]:
sns.boxplot(filtered_df['quantity'])

## рассчитаем сумму стоимости каждой товарной позиции

In [ ]:
#объеденим условия для составления общего датасета
filtered_df = df_gift_info[(df_gift_info['price'] > 0) & (df_gift_info['price'] < 3000) & (df_gift_info['quantity'] > 0) & (df_gift_info['quantity'] < 30)]

In [ ]:
filtered_df = filtered_df.copy()
filtered_df['total_cost'] = filtered_df['price'] * filtered_df['quantity']

In [ ]:
filtered_df['total_cost'].describe().apply(lambda x: f"{x:.2f}")

In [ ]:
filtered_df[filtered_df['total_cost'] > 40000]

## рассмотрим все заказы с повторояющимся id

In [ ]:
duplicates = filtered_df[filtered_df.duplicated(['order_id', 'customer_id', 'entry_date'], keep=False)].sort_values('order_id')
duplicates.head(10)

на данный момент не ясно чем вызвано дублирование данных, возможно это редактирование заказа и необходимо оставить только последнюю запись, либо это части заказа которые следует объеденить в одну или же это особенност при сборе данных. Поэтому продолжим работать с данными в первоначальном виде.

## Изучим полноту данных, анализируя время записей и посчитаем по месяцам количество дней, в которых не было продаж чтобы определить где содержиться основная часть данных

In [ ]:
min_date = filtered_df['entry_date'].min()
max_date = filtered_df['entry_date'].max()
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')

# Уникальные даты с продажами
sales_dates = filtered_df[['entry_date']].drop_duplicates()

# Создаём DataFrame со всеми датами
full_date_df = pd.DataFrame({'date': all_dates})
full_date_df['year_month'] = full_date_df['date'].dt.to_period('M')

# Помечаем даты с продажами
full_date_df['has_sales'] = full_date_df['date'].isin(sales_dates['entry_date'])

# Группируем по месяцам и считаем дни без продаж
result = full_date_df.groupby('year_month')['has_sales'].agg(
    total_days='count',
    sales_days='sum',
    no_sales_days=lambda x: (x == False).sum()
).sort_values(by='year_month', ascending=False).reset_index()

result

In [ ]:
#визуализируем результаты
plt.figure(figsize=(12, 6))
plt.bar(result['year_month'].astype(str), result['no_sales_days'])
plt.title('Количество дней без продаж по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Дней без продаж')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

по полученным результатам мы видим что в магазине в основном множество покупок в один день в следствии чего было получено так много дней без продаж

основные данные о продажах представлены за 2019 год, поэтому только с ним продолжим работать

In [ ]:
filtered_df = filtered_df[filtered_df['entry_date'] > '2019-01-01 00:00:00']

In [ ]:
filtered_df.info()

## Оценим по часам и дням недели количество заказов и количество уникальных покупателей и рассмотрим цикличность покупательской активности

In [ ]:
filtered_df['day_of_week'] = filtered_df['entry_date'].dt.day_name()  
filtered_df['hour'] = filtered_df['entry_date'].dt.hour

In [ ]:
#найдем количесво заказов и пользователей по дням недели
d = filtered_df.groupby('day_of_week').agg(orders_count=('order_id', 'count'),unique_customers=('customer_id', 'nunique'))\
.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
d

In [ ]:
#найдем количесво заказов и пользователей по часам
dh = filtered_df.groupby('hour').agg(orders_count=('order_id', 'count'),unique_customers=('customer_id', 'nunique')).sort_index()
dh

In [ ]:
# Визуализация
plt.figure(figsize=(18, 12))

# График 1: Активность по дням недели
plt.subplot(2, 2, 1)
sns.barplot(x=d.index, y='orders_count', data=d, color='royalblue', order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Общее количество заказов по дням недели', pad=20)
plt.xlabel('День недели')
plt.ylabel('Количество заказов')
plt.xticks(rotation=45)

plt.subplot(2, 2, 2)
sns.barplot(x=d.index, y='unique_customers', data=d, color='salmon', order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Уникальные покупатели по дням недели', pad=20)
plt.xlabel('День недели')
plt.ylabel('Количество покупателей')
plt.xticks(rotation=45)

# График 2: Активность по часам
plt.subplot(2, 2, 3)
sns.lineplot(x=dh.index, y='orders_count', data=dh,
             marker='o', color='royalblue', linewidth=2.5)
plt.title('Распределение заказов по часам', pad=20)
plt.xlabel('Час дня')
plt.ylabel('Количество заказов')
plt.xticks(range(0, 24))
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
sns.lineplot(x=dh.index, y='unique_customers', data=dh,
             marker='o', color='salmon', linewidth=2.5)
plt.title('Распределение покупателей по часам', pad=20)
plt.xlabel('Час дня')
plt.ylabel('Количество покупателей')
plt.xticks(range(0, 24))
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

На графике часовой активности наблюдается пик в 12 часов и резкое снижение активности, это можно выделить как цикличность активности клиентов которая растет в первой половине дня и падает к к концу.При этом на графике распределения заказов по часам наблюдается два пика в 12 часов и в 15 при условии что среднее количество покупателей к 15 часам на порядок ниже чем в 12 часов.  А по дням недели наблюдается максимальная активность клиентов в воскресенье, после чего происходит спад до минимального уровня во вторник и после чего показатели постепенно увеличиваются к началу выходных с небольшой просадкой в субботу. Это можно охарактеризовать как цикл клиентской активности, начинающийся с минимального значения во вторник, который возрастает до конца недели и уменьшающийся ко вторнику.

## Рассчитаем по месяцам среднюю выручку с клиента в день и количество уникальных покупателей

In [ ]:
filtered_df['entry_date_month'] = pd.to_datetime(filtered_df['entry_date']).dt.month

In [ ]:
#рассчитаем число уникальных клиентов и общую выручку по месяцам
stats = filtered_df.groupby('entry_date_month').agg(
    total_revenue=('total_cost', 'sum'),
    unique_users=('customer_id', 'nunique')
).reset_index()

# вычисляем среднюю выручку на клиента
stats['avg_revenue_per_user'] = stats['total_revenue'] / stats['unique_users']
stats

In [ ]:
#визуализируем результаты
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

sns.barplot(data=stats, x='entry_date_month', y='avg_revenue_per_user', ax=ax[0], color='skyblue')
ax[0].set_title('Средняя выручка с клиента по месяцам')
ax[0].set_xlabel('Месяц')
ax[0].set_ylabel('Средняя выручка')
ax[0].tick_params(axis='x', rotation=45)


sns.barplot(data=stats, x='entry_date_month', y='unique_users', ax=ax[1], color='salmon')
ax[1].set_title('Количество уникальных клиентов по месяцам')
ax[1].set_xlabel('Месяц')
ax[1].set_ylabel('Количество')
ax[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

наибольшая клиентская активность наблюдается в осенние месяцы, но не смотря на это средняя выручка с клиента в этот период увеличивается всреднем на 3000.В декабре такие низкие показатели так как данные были собраны не за весь месяц. В зимние месяцы число клиентов падает до своего минимума и после чего постепенно возрастает с каждым месяцем, но при этом средняя выручка клиентов не имеет таких просадок и находится на одном уровне

## Рассчитаем стики-фактор за второй и третий квартал 2019 года.

In [ ]:
filtered_df.head(1)

In [ ]:
# Добавим необходимые колонки
filtered_df['quarter'] = filtered_df['entry_date'].dt.quarter
filtered_df['entry_day'] = filtered_df['entry_date'].dt.date
filtered_df['entry_date_month'] = pd.to_datetime(filtered_df['entry_date']).dt.month

# Расчет DAU - группируем по полной дате (entry_date)
dau = filtered_df.groupby('entry_day').agg(
    count_users=('customer_id', 'nunique'),
    quarter=('quarter', 'first')
).reset_index()

# Расчет MAU - группируем по месяцу (entry_date_month)
mau = filtered_df.groupby('entry_date_month').agg(
    count_users=('customer_id', 'nunique'),
    quarter=('quarter', 'first')
).reset_index()

# Средние значения для кварталов
avg_dau_q2 = dau[dau['quarter'] == 2]['count_users'].mean()
avg_dau_q3 = dau[dau['quarter'] == 3]['count_users'].mean()
avg_mau_q2 = mau[mau['quarter'] == 2]['count_users'].mean()
avg_mau_q3 = mau[mau['quarter'] == 3]['count_users'].mean()

sticky_q2 = avg_dau_q2 / avg_mau_q2
sticky_q3 = avg_dau_q3 / avg_mau_q3

print(f"Стики-фактор для Q2: {sticky_q2:.4f}")
print(f"Стики-фактор для Q3: {sticky_q3:.4f}")

Стики фактор показывает как регульрно клиенты пользуются услугами нашего магазина и по результатам за 2 и 3 кварталы 2019 года мы видим что в среднем около 5% пользователей пользуются нашими услугами ежедневно

In [ ]:
dau = filtered_df.groupby('entry_day').agg(
    count_users=('customer_id', 'nunique'),
    quarter=('quarter', 'first')
).reset_index()
dau

In [ ]:
dau['quarter'].value_counts()

## Составим профиль каждого клиента с информацией о количестве заказов, дате первого и последнего заказа, общей суммуе всех заказов, средней цену заказа, первом заказанном товаре

In [ ]:
profiles = filtered_df.sort_values(by=['customer_id', 'entry_date', 'total_cost', 'order_id'])

In [ ]:
profiles = profiles.groupby('customer_id').agg(orders_count = ('order_id', 'nunique'),\
                                    min_date = ('entry_date', 'min'),\
                                    max_date = ('entry_date', 'max'),\
                                    total_sum_orders = ('total_cost', 'sum'),           
                                    first_product = ('entry', 'first')).reset_index()

#рассчитаем среднюю стоимость заказа
profiles['mean_cost_orders'] = profiles['total_sum_orders'] / profiles['orders_count']
profiles

## Разделим клиентов на возвратных и нет по признаку наличия повторных покупок и для каждой из групп рассчитаем средние показатели

In [ ]:
returning_customers = profiles[profiles['orders_count'] > 1]  # Возвратные
one_time_customers = profiles[profiles['orders_count'] == 1]  # Невозвратные

In [ ]:
#функция для расчета средних показателей в группах
def analyze_group(df, group_name):
    print(f"\nАнализ группы: {group_name}")
    print(f"Количество клиентов: {len(df)}")
    print("Средние показатели:")
    print(f"- Число заказов: {df['orders_count'].mean():.2f}")
    print(f"- Общая сумма заказов: {df['total_sum_orders'].mean():.2f}")
    print(f"- Средний чек: {df['mean_cost_orders'].mean():.2f}")
    print(f"- Время между первым и последним заказом (дней): {(df['max_date'] - df['min_date']).dt.days.mean():.2f}")

# Применяем к обеим группам
analyze_group(returning_customers, "Возвратные клиенты")
analyze_group(one_time_customers, "Невозвратные клиенты")

около 60% клиентов возвращаются за нашими услугами и в среднем делают около 5 заказов и пользуются сервисом около полу года, также стоит заметить что средний чек в двух группах отличается всего на 1000 рублей

In [ ]:
#рассмотрим топ 5 популярных товаров у клиентов 
returning_customers['first_product'].value_counts().head(5)

## Проведем RFM-сегментацию клиентов 

In [ ]:
filtered_df['entry_date'].max()

предположим что анализ начал проводиться 11 декабря 2019 года

In [ ]:
#рассчитаем разницу между последней покупкой клиента и началом анализа
filtered_df['orders_recency'] = (pd.to_datetime('2019-12-11') - filtered_df.groupby('customer_id')['entry_date'].transform('max')).dt.days

In [ ]:
filtered_df.head()

In [ ]:
#рассчитаем необходимые метрики
rfm = filtered_df.groupby('customer_id').agg(recency =('orders_recency', 'min'),\
                                          frequency = ('order_id', 'nunique'),\
                                          monetary = ('total_cost', 'sum')).reset_index()
rfm.head()

In [ ]:
#разделим данные по группам
rfm['r'] = pd.qcut(rfm['recency'], q=3, labels=[3,2,1])
rfm['f'] = pd.cut(rfm['frequency'], bins=[0,1,10,np.inf], labels=[1,2,3])
rfm['m'] = pd.qcut(rfm['monetary'], q=3, labels=[1,2,3])

# Найдем групповой RFM индекс:
rfm[['r','f','m']] = rfm[['r','f','m']].astype('str')
rfm['rfm_group'] = rfm['r'] + rfm['f'] + rfm['m']

# Найдем сумму индексов RFM:
rfm[['r','f','m']] = rfm[['r','f','m']].astype('int')
rfm['rfm_sum'] = rfm[['r','f','m']].sum(axis=1)
rfm

In [ ]:
# Построим график treemap для визуализации результатов RFM сегментации:
fig = px.treemap(rfm,
                 path=['rfm_group'], # Выбираем RFM-сегменты
                 values='customer_id', # Устанавливаем размер - количество покупателей
                 color='rfm_sum', # Цвет сегмента будет определять сумма RFM
                 color_continuous_scale='Sunset',
                 title='RFM сегментация пользователей')

# Отобразим график:
fig.show()

In [ ]:
#рассмотрим получившиеся группы
rfm['rfm_group'].value_counts()

большинство клиентов попало в группу 111, от которой следует либо отказаться либо реанимировать

некоторое способы работы с данными группами:

- 111, 211, 311 Увеличить частоту покупок и средний чек. В этом помогут скидки за объём покупок и накопительные программы лояльности.
- 112 Доходность на среднем уровне, но покупали давно и нечасто. Стоит применить стратегию реактивации — возвращающие письма, акции, промокоды.
- 212, 312, 313, 213 Нужно увеличить частоту покупок. Стратегия — акции и скидки за регулярность покупок.
- 333 «Золотой сегмент». Необходимо предпринять действия по удержанию: программы лояльности, индивидуальное обслуживание.
- 323, 233, 223 Выгодные сегменты. Необходимо предпринять действия по удержанию: программы лояльности, индивидуальное обслуживание.
- 123 Клиенты покупали достаточно давно, но на большую сумму и со средней частотой. Стратегия — реактивация: возвращающие письма, акции, промокоды.
- 113 Клиенты покупали достаточно давно и нечасто, но на большую сумму. Стратегия — реактивация: возвращающие письма, акции, промокоды.
- 133 Клиенты покупали достаточно давно, но часто и на большую сумму. Стратегия — реактивация: возвращающие письма, акции, промокоды.
- 122 Клиенты покупали достаточно давно, но доходность и частота на среднем уровне. Стратегия — реактивация: возвращающие письма, акции, промокоды.
- 322, 222 Стабильные сегменты. Необходимо предпринять действия по удержанию: программы лояльности, индивидуальное обслуживание.

## Сравним доли возвратных и невозвратных клиентов за второй и третий квартал 2019 года

Проверим нулевую гипотезу что между долями возвратных и невозвратных клиентов нет значимых различий.Альтернативная гипотеза будет заключаться что между ними есть значимые отличия.

In [ ]:
filtered_df.columns

In [ ]:
filtered_df.head(1)

In [ ]:
#выделим необходимые временные периоды
q2 = filtered_df[filtered_df['quarter'] == 2]
q3 = filtered_df[filtered_df['quarter'] == 3]

In [ ]:
#число вернувшихся и невернувшихся клиентов во 2 квартале
return_q2 = q2.groupby('customer_id')['order_id'].nunique().reset_index(name='orders_count').query("orders_count > 1").shape[0]
print(return_q2)

unreturn_q2 = q2.groupby('customer_id')['order_id'].nunique().reset_index(name='orders_count').query("orders_count == 1").shape[0]
print(unreturn_q2)

#общее число клиентов 
sum_q2 = return_q2 + unreturn_q2
print(sum_q2)

In [ ]:
#число вернувшихся и невернувшихся клиентов в 3 квартале
return_q3 = q3.groupby('customer_id')['order_id'].nunique().reset_index(name='orders_count').query("orders_count > 1").shape[0]
print(return_q3)

unreturn_q3 = q3.groupby('customer_id')['order_id'].nunique().reset_index(name='orders_count').query("orders_count == 1").shape[0]
print(unreturn_q3)

#общее число клиентов
sum_q3 = return_q3 + unreturn_q3
print(sum_q3)

так как нужно узать есть ли значимая разница между числом конверсий, то воспользуемся z-тестом

In [ ]:
alpha = 0.05
successes = [796, 887]
nobs = [1957, 2098]

z_stat, p_value = proportions_ztest(count=successes, nobs=nobs)
print(f"P-value: {p_value}")

if (p_value < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

По результатам тестирования было выяснено что между размерами долей возвратных и невозвратных клиентов за 2 и 3 квартал 2019 года нет значимых различий 

## Сравним средние чеки в странах с country_id, равному 3, 6 и 24.

Проверим гипотезу о том что средние чеки в странах 3, 6 и 24 не отличаются.Альтернативная гипотеза будет гласить что различия есть.

In [ ]:
filtered_df.columns

In [ ]:
#колво заказов
filtered_df['orders_count'] = filtered_df.groupby('customer_id')['order_id'].transform('count')

#сумма всех заказов
filtered_df['total_sum_orders']  = filtered_df.groupby('customer_id')['total_cost'].transform('sum')

In [ ]:
#добавим столбец со средним чеком 
filtered_df['mean_cost_orders'] = filtered_df['total_sum_orders'] / filtered_df['orders_count']

для того чтобы определить какой тест необходимо использовать, узнаем есть ли нормальность распределения и равенство дисперсий между выборок

In [ ]:
#нормальность распределений
shap_3 = shapiro(filtered_df[filtered_df['country_id']== 3]['mean_cost_orders'])
print(shap_3)

shap_6 = shapiro(filtered_df[filtered_df['country_id'] == 6]['total_cost'])
print(shap_6)

shap_24 = shapiro(filtered_df[filtered_df['country_id'] == 24]['total_cost'])
print(shap_24)

In [ ]:
#равенство дисперсий
_, p_levene = levene(filtered_df[filtered_df['country_id']== 3]['mean_cost_orders'],\
                filtered_df[filtered_df['country_id'] == 6]['total_cost'],\
                filtered_df[filtered_df['country_id'] == 24]['total_cost'])
print(f'p-value теста Левена: {p_levene:.3f}')

тк у выборок отсутствует нормальность распределения и равенство дисперсий, воспользуемся непараметрическим тестом Крускала-Уолиса

In [ ]:
alpha = 0.05
h_stat, p_kruskal = kruskal(
    filtered_df[filtered_df['country_id'] == 3]['total_cost'],
    filtered_df[filtered_df['country_id'] == 6]['total_cost'],
    filtered_df[filtered_df['country_id'] == 24]['total_cost']
)
print(f'Результат Крускала-Уоллиса: H = {h_stat:.2f}, p-value = {p_kruskal:.3f}')

if (p_kruskal < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

тк р-значение больше 0.05 можно сделать вывод о том что статистические значимой разницы между средними нет 

In [ ]:
filtered_df.columns

## Проверим гипотезу о том что клиенты, совершившие первую покупку в выходные имею большее LTV 

Нулевая гипотеза заключается в том что выручка с клиента за все его время жизни, который совершил первую покупку в выходной день не отличается от клиента сделавшего первую покупку в будний день. Альтернативная будет гласить что у первого типа клиентов выручка больше.

In [ ]:
# Найдем дату первого заказа каждого клиента
filtered_df['first_order_date'] = filtered_df.groupby('customer_id')['entry_date'].transform('min')
filtered_df['first_order_day'] = filtered_df['first_order_date'].dt.day_name()

In [ ]:
# Размечаем клиентов по типу первого заказа
filtered_df['weekend_first'] = filtered_df['first_order_day'].isin(['Saturday', 'Sunday'])

# Расчет LTV (общая выручка от клиента)
ltv = filtered_df.groupby('customer_id').agg(
    ltv=('total_cost', 'sum'),
    first_order_day=('first_order_day', 'first')
).reset_index()
ltv

In [ ]:
#разделим клиентов на группы
weekend_ltv = ltv[ltv['first_order_day'].isin(['Saturday', 'Sunday'])]['ltv']
weekday_ltv = ltv[~ltv['first_order_day'].isin(['Saturday', 'Sunday'])]['ltv']

print(f"Средний LTV (выходные): {weekend_ltv.mean():.2f}")
print(f"Средний LTV (будни): {weekday_ltv.mean():.2f}")

plt.figure(figsize=(10, 6))
sns.boxplot(x='first_order_day', y='ltv', 
            data=ltv, 
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.title('Распределение LTV по дню первого заказа')
plt.show()

In [ ]:
# Проверка нормальности
_, p_weekend = shapiro(weekend_ltv)
_, p_weekday = shapiro(weekday_ltv)
print(f"Тест Шапиро (выходные): p-value = {p_weekend:.3f}")
print(f"Тест Шапиро (будни): p-value = {p_weekday:.3f}")

тк у выборок нет нормального распределения используем U-тест Манна-Уитни

In [ ]:
stat, p_value = mannwhitneyu(weekend_ltv, weekday_ltv, alternative='greater')
print(f"p-value = {p_value:.4f}")

if p_value < 0.05:
    print("Отвергаем нулевую гипотезу: LTV клиентов с первым заказом в выходные значимо выше")
else:
    print("Нет оснований отвергать нулевую гипотезу")

In [ ]:
filtered_df['customer_id'].nunique()

# Вывод

В ходе проделанной работы была выполнена предобработка данных, определен временной период в котором хранится большинство данных, а именно 2019 год.Проведен анализ покупательской активности клиентов магазина для выявления цикличности, расчитан стики-фактор за 2 и 3 квартал 2019 года, а также составлены профили клиентов и на их основе пользователи были разделены на возвратных и нет. Была проведена rfm сегментация клиентов и предложены мероприятия по работе с каждой из групп. На последнем этапе были проверены 3 гипотезы при помощи статистических тестов.
По итогам работы были получены следующие результаты:
- Наибольшая клиентская активность наблюдается в воскреснье, в этот день услугами магазина пользуются в среднем 1687 клиентов, которые делают 55913 заказов. Самый непопулярный день недели это вторник - в среднем около 657 клиентов, совершающих 18686 заказов. В остальные дни показатели находятся на уровне от 1300 до 1500 уникальных клиентов в день и от 40000 до 50000 заказов. При анализе почасовой активности пользователей было выявлено что наибольшее число клиентов пользуются услугами магазина в полдень - в среднем 1458 человек, после чего их число постепенно уменьшается, при этом число заказов имеет два максимума - в 12 и 15 часов с общим числом заказов 44691 и 45583 соответственно.
- При анализе средней выручки с клиента в день по месяцам было определено, что наибольшее число клиентов приходит в осенние месяцы, а именно от 1116 до 1389 при среднем количестве в районе 900, но при этом средняя выручка с клиента  в этот период практически не изменяется со среднего значения 32879 она увеличивается до 35684.
- стики-фактор рассчитанный за 2 и 3 квартал 2019 года составил около 5%, такой низкий показатель можно объяснить тем, что магазин специализируется на подарочных товарах и в следствии чего у клиентов нет необходимости дарить что-то каждый день.
- На основе составленных профилей клиентов была проведена сегментация по признаку наличия повторных покупок и были рассчитаны средние показатели полученных групп. Число возвратных клиентов составило 2542, а невозвратных 1489. Возвратные клиенты в среднем соверщают около 6 покупок со средним чеком 20624, клиенты совершившие единоразовую покупку имеют средний чек 19564. 
- После проведения сегментации клиентов по признакам:давность покупки, частота покупок и суммарной стоимости всех покупок было выявлено что из 4031 пользователя 589 являются самыми невыгодными по давности, частоте и общей сумме заказов, при этом, это самая многочисленная группа, над которой следует работать чтобы повысить ее показатели. Число самых выгодных клиентов составило 203, этих клиентов следует удерживать как можно дольше и повышать их показатели. В процессе работы были предоставлены рекомендации по работе с каждой группой.
- Проверка гипотез о статистической значимости различий между долями возвратных и невозвратных клиентов за второй и третий квартал 2019 года, различий между средними чеками в странах с country_id, равному 3, 6 и 24, а также что клиенты совершившие первую покупку в выходные имею большее LTV, не подтвердилась.


# Анализ товаров которые чаще всего покупают вместе

In [ ]:
# Группируем товары по заказам (учитываем quantity)
orders = filtered_df.groupby('order_id').apply(lambda x: list(x['entry'].repeat(x['quantity'])))

# Словарь для подсчета совместных покупок
pair_counts = defaultdict(int)

# Перебираем все заказы
for items in orders:
    # Получаем уникальные товары в заказе 
    unique_items = list(set(items))
    
    # Генерируем все возможные пары в заказе
    for pair in combinations(sorted(unique_items), 2):
        pair_counts[pair] += 1
        
# Преобразуем в DataFrame
pairs_df = pd.DataFrame.from_dict(pair_counts, orient='index', columns=['frequency'])
pairs_df.index = pd.MultiIndex.from_tuples(pairs_df.index, names=['item1', 'item2'])

# Сортируем по частоте
pairs_df = pairs_df.sort_values('frequency', ascending=False)

pairs_df.head(10)